## Práctico Final Planning 2020  - Segunda Parte


### Imports

In [1]:
import argparse
import time
import msgpack
from enum import Enum, auto
from queue import PriorityQueue
import numpy as np

from planning_utils import a_star, heuristic, create_grid, read_global_home, prune_path
from udacidrone import Drone
from udacidrone.connection import MavlinkConnection
from udacidrone.messaging import MsgID
from udacidrone.frame_utils import global_to_local


from shapely.geometry import Point
import random
import time
import csv
from shapely.geometry import Polygon
from sklearn.neighbors import KDTree
import networkx as nx
import numpy.linalg as LA
from shapely.geometry import Polygon, Point, LineString
from skimage.draw import *

In [2]:
class States(Enum):
    MANUAL = auto()
    ARMING = auto()
    TAKEOFF = auto()
    WAYPOINT = auto()
    LANDING = auto()
    DISARMING = auto()
    PLANNING = auto()


In [3]:
def create_polygons(data, safety_distance):
    # Valor maximo y minimo de Norte
    north_min = np.floor(np.min(data[:, 0] - data[:, 3]))
    north_max = np.ceil(np.max(data[:, 0] + data[:, 3]))

    # Valor maximo y minimo de Este
    east_min = np.floor(np.min(data[:, 1] - data[:, 4]))
    east_max = np.ceil(np.max(data[:, 1] + data[:, 4]))

    # Calculamos el ancho y alto del grid con
    # los valores maximos y minimos, considerando que
    # discretizamos a una unidad
    north_size = int(np.ceil(north_max - north_min))
    east_size = int(np.ceil(east_max - east_min))

    polygons = []
    alt_poly = []
    # Buscamos los polygonos
    for i in range(data.shape[0]):
        north, east, alt, d_north, d_east, d_alt = data[i, :]
        obstacle = [
            int(np.clip(north - d_north - safety_distance - north_min, 0, north_size - 1)),
            int(np.clip(north + d_north + safety_distance - north_min, 0, north_size - 1)),
            int(np.clip(east - d_east - safety_distance - east_min, 0, east_size - 1)),
            int(np.clip(east + d_east + safety_distance - east_min, 0, east_size - 1)),
        ]
        x1 = (obstacle[2], obstacle[0])
        x2 = (obstacle[2], obstacle[1])
        y2 = (obstacle[3], obstacle[0])
        y1 = (obstacle[3], obstacle[1])
        coords = [x1, x2, y1, y2]
        poly = Polygon(coords)
        polygons.append(poly)
        alt_poly.append(alt + d_alt)

    return polygons, alt_poly, north_size, east_size





def heuristic2(n1, n2):
    return LA.norm(np.array(n2) - np.array(n1))


def a_star2(graph, heuristic, start, goal):
    path = []
    queue = PriorityQueue()
    queue.put((0, start))
    visited = set(start)

    branch = {}
    found = False

    while not queue.empty():
        item = queue.get()
        current_cost = item[0]
        current_node = item[1]

        if current_node == goal:
            print('Encontrado Path')
            found = True
            break
        else:
            for next_node in graph[current_node]:
                cost = graph.edges[current_node, next_node]['weight']
                new_cost = current_cost + cost + heuristic2(next_node, goal)

                if next_node not in visited:
                    visited.add(next_node)
                    queue.put((new_cost, next_node))

                    branch[next_node] = (new_cost, current_node)

    path = []
    path_cost = 0
    if found:

        # retrace steps
        path = []
        n = goal
        path_cost = branch[n][0]
        while branch[n][1] != start:
            path.append(branch[n][1])
            n = branch[n][1]
        path.append(branch[n][1])

    return path[::-1], path_cost

        
def can_connect(n1, n2, grid):
    rr, cc = line(int(n1[1]),int(n1[0]),int(n2[1]),int(n2[0]))
    for i in range(len(rr)):
        if (grid[rr[i]][cc[i]] == 1):
            return False
    return True

def create_graph(nodes, k, grid):
    g = nx.Graph()
    tree = KDTree(nodes)
    for n1 in nodes:
        idxs = tree.query([n1], k, return_distance=False)[0]

        for idx in idxs:
            n2 = tuple(nodes[idx])
            n1 = tuple(n1)
            if (n2 == n1):
                continue

            if can_connect(n1, n2, grid):
                g.add_edge(n1, n2, weight=1)
    return g

In [4]:
class MotionPlanning(Drone):

    def __init__(self, connection):
        super().__init__(connection)

        self.target_position = np.array([0.0, 0.0, 0.0])
        self.waypoints = []
        self.in_mission = True
        self.check_state = {}

        # initial state
        self.flight_state = States.MANUAL

        # register all your callbacks here
        self.register_callback(MsgID.LOCAL_POSITION, self.local_position_callback)
        self.register_callback(MsgID.LOCAL_VELOCITY, self.velocity_callback)
        self.register_callback(MsgID.STATE, self.state_callback)

    def local_position_callback(self):
        if self.flight_state == States.TAKEOFF:
            if -1.0 * self.local_position[2] > 0.95 * self.target_position[2]:
                self.waypoint_transition()
        elif self.flight_state == States.WAYPOINT:
            if np.linalg.norm(self.target_position[0:2] - self.local_position[0:2]) < 1.0:
                if len(self.waypoints) > 0:
                    self.waypoint_transition()
                else:
                    if np.linalg.norm(self.local_velocity[0:2]) < 1.0:
                        self.landing_transition()

    def velocity_callback(self):
        if self.flight_state == States.LANDING:
            if self.global_position[2] - self.global_home[2] < 0.1:
                if abs(self.local_position[2]) < 0.01:
                    self.disarming_transition()

    def state_callback(self):
        if self.in_mission:
            if self.flight_state == States.MANUAL:
                self.arming_transition()
            elif self.flight_state == States.ARMING:
                if self.armed:
                    self.plan_path()
            elif self.flight_state == States.PLANNING:
                self.takeoff_transition()
            elif self.flight_state == States.DISARMING:
                if ~self.armed & ~self.guided:
                    self.manual_transition()

    def arming_transition(self):
        self.flight_state = States.ARMING
        print("arming transition")
        self.arm()
        self.take_control()

    def takeoff_transition(self):
        self.flight_state = States.TAKEOFF
        print("takeoff transition")
        self.takeoff(self.target_position[2])

    def waypoint_transition(self):
        self.flight_state = States.WAYPOINT
        print("waypoint transition")
        self.target_position = self.waypoints.pop(0)
        print('target position', self.target_position)
        self.cmd_position(self.target_position[0], self.target_position[1], self.target_position[2], self.target_position[3])

    def landing_transition(self):
        self.flight_state = States.LANDING
        print("landing transition")
        self.land()

    def disarming_transition(self):
        self.flight_state = States.DISARMING
        print("disarm transition")
        self.disarm()
        self.release_control()

    def manual_transition(self):
        self.flight_state = States.MANUAL
        print("manual transition")
        self.stop()
        self.in_mission = False

    def send_waypoints(self):
        print("Sending waypoints to simulator ...")
        data = msgpack.dumps(self.waypoints)
        self.connection._master.write(data)

    def plan_path(self):
        self.flight_state = States.PLANNING
        print("Searching for a path ...")
        TARGET_ALTITUDE = 5
        SAFETY_DISTANCE = 1

        self.target_position[2] = TARGET_ALTITUDE

        lat0, lon0 = read_global_home('colliders.csv')
        
        self.set_home_position(lon0, lat0, 0)
        
        print('global position {}', format(self.global_position))
 
        local_north, local_east, _ =global_to_local(self.global_position, self.global_home)
        
        print('global home {0}, position {1}, local position {2}'.format(self.global_home, self.global_position,
                                                                         self.local_position))
        
        inicio=time.time()
        
        data = np.loadtxt('colliders.csv', delimiter=',', dtype='Float64', skiprows=2)
  
        grid, north_offset, east_offset = create_grid(data, TARGET_ALTITUDE, SAFETY_DISTANCE)
        print("North offset = {0}, east offset = {1}".format(north_offset, east_offset))
        
        grid_start_north = int(np.ceil(local_north - north_offset))
        grid_start_east = int(np.ceil(local_east - east_offset))
        grid_start = (grid_start_north, grid_start_east)
        print(grid_start)

        lat_goal, long_goal = 37.795757, -122.400162
        goal_global_position = np.array([long_goal, lat_goal, 0.0 ])
        
        local_north_goal, local_east_goal, _ = global_to_local(goal_global_position, self.global_home)
        
        grid_goal_north = int(np.ceil(local_north_goal - north_offset))
        grid_goal_east = int(np.ceil(local_east_goal - east_offset))
        grid_goal = (grid_goal_north, grid_goal_east)
        
        goal=[grid_goal_east, grid_goal_north]
        start= [ grid_start_east, grid_start_north]
        print('Local Start and Goal: ', grid_start, grid_goal)
        print(grid[grid_goal_north][grid_goal_east])
        
        #PRM
        
        TARGET_ALTITUDE = 5
        SAFETY_DISTANCE = 2
        
        grid, north_offset, east_offset = create_grid(data, TARGET_ALTITUDE, SAFETY_DISTANCE)

        # Generamos puntos random
        N_PUNTOS = 500
        puntos = []
        puntos_generados=0
        while puntos_generados < N_PUNTOS:
            x=random.randint(0,(len(grid[0])-1))
            y=random.randint(0,(len(grid[0])-1))
            if grid[y,x] == 0:
                puntos.append(Point(x,y))
                puntos_generados+=1
    
        puntos_filtrados = puntos
        
        puntos_filtrados_list = []
        for i in puntos_filtrados:
            puntos_filtrados_list.append([i.x,i.y])
        puntos_filtrados_list.append(start)
        puntos_filtrados_list.append(goal)
        
        grafo = create_graph(np.asarray(puntos_filtrados_list), 10, grid)
       
        start = tuple(start)
        goal = tuple(goal)
        path, cost = a_star2(grafo, heuristic2, start, goal)
        if not path:
            print('NO SE ENCONTRO CAMINO')
        
        path.append(list(goal))
        
        final=time.time()
        
        demora=final-inicio
        print('TIEMPO EMPLEADO = ', demora)
        
        
        waypoints = [[int(p[1]) + north_offset, int(p[0]) + east_offset, TARGET_ALTITUDE, 0] for p in path]
        self.waypoints = waypoints
        self.send_waypoints()

    def start(self):
        self.start_log("Logs", "NavLog.txt")

        print("starting connection")
        self.connection.start()

        # Only required if they do threaded
        # while self.in_mission:
        #    pass

        self.stop_log()


In [5]:
#if __name__ == "__main__":
parser = argparse.ArgumentParser()

In [6]:
parser.add_argument('--port', type=int, default=5760, help='Port number')

_StoreAction(option_strings=['--port'], dest='port', nargs=None, const=None, default=5760, type=<class 'int'>, choices=None, help='Port number', metavar=None)

In [7]:
parser.add_argument('--host', type=str, default='127.0.0.1', help="host address, i.e. '127.0.0.1'")

_StoreAction(option_strings=['--host'], dest='host', nargs=None, const=None, default='127.0.0.1', type=<class 'str'>, choices=None, help="host address, i.e. '127.0.0.1'", metavar=None)

In [8]:
conn = MavlinkConnection('tcp:127.0.0.1:5760', timeout=300000000)

In [9]:
drone = MotionPlanning(conn)

Logs\TLog.txt


In [10]:
time.sleep(1)

In [11]:
drone.start()

Logs\NavLog.txt
starting connection
arming transition
Searching for a path ...
lat0 37.792480, lon0 -122.397450

global position {} [-1.22397450e+02  3.77924799e+01  9.30000000e-02]
global home [-122.39745   37.79248    0.     ], position [-1.22397450e+02  3.77924799e+01  9.30000000e-02], local position [-0.00070054  0.03462752 -0.0935117 ]
1608249076.2339945
North offset = -316, east offset = -445
(316, 446)
Local Start and Goal:  (316, 446) (679, 204)
0.0
1
2
502
3
Encontrado Path
TIEMPO EMPLEADO =  16.268496990203857
[(446, 316), (435.0, 397.0), (416.0, 456.0), (418.0, 509.0), (403.0, 574.0), (351.0, 581.0), (299.0, 608.0), (262.0, 633.0), (276.0, 675.0), (223.0, 693.0), [204, 679]]
[[0, 1, 5, 0], [81, -10, 5, 0], [140, -29, 5, 0], [193, -27, 5, 0], [258, -42, 5, 0], [265, -94, 5, 0], [292, -146, 5, 0], [317, -183, 5, 0], [359, -169, 5, 0], [377, -222, 5, 0], [363, -241, 5, 0]]
Sending waypoints to simulator ...
takeoff transition
waypoint transition
target position [0, 1, 5, 0]
way